In [1]:
#pip install selenium

In [11]:
# import libraries 

from bs4 import BeautifulSoup
from selenium import webdriver

In [12]:
driver = webdriver.Chrome()

In [13]:
url = "https://www.walmart.ca/search/pantry%20food/N-117/page-1"
driver.get(url)

In [14]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [15]:
results = soup.find_all('div', {'data-automation' : 'grocery-product'})

In [16]:
len(results)

60

In [36]:
item = results[0]

In [48]:
description = item.find('class', {'data-automation', 'description'})

In [49]:
print(description)

None


In [92]:
name = item.p.text
name

'Grace Kennedy Corn Meal'

In [93]:
current_price = item.a.span.text
current_price

'$3.77'

In [94]:
unit_price = item.find('span', 'price-per-unit')
print(unit_price)

None


In [91]:
description = item.find('p','css-1m0dajq eudvd6x0').text
description

'2 kg'

In [97]:
def extract_records(item):
    #name
    name = item.p.text
    
    #current price
    try:
        current_price = item.a.span.text
    except AttributeError:
        current_price =''
    #unit_price
    try:
        unit_price = item.find('span', 'price-per-unit').text
    except AttributeError:
        return
    #description
    try:
        description = item.find('p','css-1m0dajq eudvd6x0').text
    except AttributeError:
        description =''
    result = (name, current_price, unit_price, description)
    
    return result
    

In [98]:
records = []
results = soup.find_all('div', {'data-automation' : 'grocery-product'})

for item in results:
    records.append(extract_records(item))

In [102]:
import pandas as pd

In [103]:
df = pd.DataFrame(records, columns =['name', 'current_price', 'unit_price', 'description'])

In [104]:
df.head()

,name,current_price,unit_price,description
0,Great Value Sliced Mushrooms,97¢,34¢/100ml,284 mL
1,"Aylmer Tomatoes Diced, No Salt Added",3 for $6,31¢/100ml,796 ml
2,Great Value Whole Mushrooms,97¢,34¢/100ml,284 mL
3,Green Giant Canned Harvest Select No Added Sal...,4 for $6,58¢/100ml,Green Giant Canned Harvest Select
4,Heinz Original Beans in Tomato Sauce,3 for $4,39¢/100ml,398mL


In [105]:
df.to_csv('data.csv', index=False)

# Get the Next Page

In [111]:
def get_url(search_term):
    """ Generate a url from search term"""
    template = 'https://www.walmart.ca/search?q={}&c=10019'
    search_term = search_term.replace(' ','+')

    
    url = template.format(search_term)
    
    #add page query place holder
    url += '&page={}'
    
    return url


# Putting Everything Together

In [120]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import csv

#function to get URL
def get_url(search_term):
    """ Generate a url from search term"""
    template = 'https://www.walmart.ca/search?q={}&c=10019'
    search_term = search_term.replace(' ','+')

    
    url = template.format(search_term)
    
    #add page query place holder
    url += '&page={}'
    
    return url

#fucntion to extract records
def extract_records(item):
    #name
    name = item.p.text
    
    #current price
    try:
        current_price = item.a.span.text
    except AttributeError:
        current_price =''
    #unit_price
    try:
        unit_price = item.find('span', 'price-per-unit').text
    except AttributeError:
        return
    #description
    try:
        description = item.find('p','css-1m0dajq eudvd6x0').text
    except AttributeError:
        description =''
    result = (name, current_price, unit_price, description)
    
    return result

#create a main function
def main(search_term):
    """Runs main program routine"""
    #startup webdriver
    driver = webdriver.Chrome()
    
    records = []
    url = get_url(search_term)
    
    for page in range(1,25):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-automation' : 'grocery-product'})
        
        for item in results:
            records.append(extract_records(item))
    
    driver.close()
    
    #save data to CSV
    with open('results.csv', 'w',newline = '',encoding ='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['name', 'current_price', 'unit_price', 'description'])
        writer.writerows(records)

In [121]:
main('pantry food')

In [124]:
data1 = pd.read_csv('results.csv')

In [126]:
data1.head(5)

,name,current_price,unit_price,description
